# Project - 모델 성능 개선하기

전처리나 형태소 분석기를 바꾸어보고, 다른 날짜의 훈련 데이터를 일부 추가해 모델을 일반화하기 위한 고민을 해봅시다.


## Step 1. 형태소 분석기 변경해보기

[앞서 진행했던 프로젝트](https://github.com/jiyeoon/aiffel/blob/master/Week2/Exploration3-NewsCrawling.ipynb)에서는 **Mecab**을 사용했습니다. 

하지만 konlpy 패키지 안에는 Mecab 외에도 Hannanum, Kkma, Komoran, Okt라는 형태소 분석기가 존재합니다.이들 형태소 분석기 중 어떤 것을 선택하는 것이 좋을지, 형태소 분석 기간의 장단점은 무엇인지, 처리 속도와 성능에는 어떤 변화가 있는지 비교해봅시다.


- 참고 : [KoNLPy 토크나이저 클래스](https://konlpy.org/ko/v0.5.2/api/konlpy.tag/)



In [2]:
import pandas as pd

df = pd.read_csv("./data/news_data.csv")
df.head(10)

,news,code
0,파주시청. 사진제공=파주시 파주시청. 사진제공=파주시\n\n[파주=파이낸셜뉴스 강근...,사회
1,동영상 뉴스\n\n이천 물류창고 화재 발화지점으로 지목된 지하 2층에서 산소절단기의...,사회
2,황범순 의정부시 부시장 을지대학교 의정부캠퍼스 및 부속병원 공사현장 안전점검. 사진...,사회
3,귀갓길 여성을 쫓아가 성범죄를 시도한 20대 남성이 구속됐습니다.서울 강남경찰서는 ...,사회
4,(서울=연합뉴스) 대한약사회가 6일부터 코로나바이러스 감염증 대응 체계를 '사회적 ...,사회
5,질서정연 코로나19 확산 방지를 위한 ‘물리적 거리 두기’가 ‘생활 속 거리 두기’...,사회
6,“코로나19에 걸렸다 나은 친구는 아무래도 좀 멀리하게 될 것 같아요. 재발 가능성...,사회
7,1977년 메이저리그 LA 다저스의 시즌 마지막 경기였다. 신인 타자 더스티 베이커...,사회
8,"6일 등교수업을 앞둔 경북 한 학교의 보건실에는 손소독제, 마스크 등 방역물품이 상...",사회
9,경찰 로고./뉴스1 © News1 신채린 기자 경찰 로고./뉴스1 © News1 신...,사회


In [3]:
# 데이터 전처리
df['news'] = df['news'].str.replace("[^ㄱ-ㅎ ㅏ-ㅣ 가-힣]", "")
df['news']

0       파주시청 사진제공파주시 파주시청 사진제공파주시파주파이낸셜뉴스 강근주 기자 파주시는 ...
1       동영상 뉴스이천 물류창고 화재 발화지점으로 지목된 지하 층에서 산소절단기의 산소 공...
2       황범순 의정부시 부시장 을지대학교 의정부캠퍼스 및 부속병원 공사현장 안전점검 사진제...
3       귀갓길 여성을 쫓아가 성범죄를 시도한 대 남성이 구속됐습니다서울 강남경찰서는 강간상...
4       서울연합뉴스 대한약사회가 일부터 코로나바이러스 감염증 대응 체계를 사회적 거리두기에...
                              ...                        
5119    신종 코로나바이러스 감염증코로나 사태 이후 가정의 달 월에도 언택트비대면 신풍속도가...
5120    는 소비자로부터 월 이용료 만만원을 받고 초고속 인터넷을 제공한다 그런 브로드밴드가...
5121    머리를 긁고 있는 오랑우탄 몸을 긁는 행동을 따라 하는 것은 부정적 감정과 관련이 ...
5122    가 오는 일 정식 출시하는 스마트폰 벨벳이 사실상 공짜폰이 될 전망이다 단말기 가격...
5123    이미지제공게티이미지뱅크 이미지제공게티이미지뱅크  전자신문  전자신문인터넷 무단전재 ...
Name: news, Length: 5124, dtype: object

In [4]:
# 중복제거
df.drop_duplicates(subset = ['news'], inplace = True)
print("중복 제거 후 뉴스 기사의 개수 : ", len(df))

중복 제거 후 뉴스 기사의 개수 :  3994


### 1.1 Hannanum 사용하기

가장 먼저 Hannanum 형태소 분석기를 사용하여 분석해봅시다.

Konlpy의 경우 JAVA로 작성된 모듈을 로드해야하기 때문에 JAVA 1.7 이상이 설치되어야 합니다. 

오라클(Oracle) 사이트에서 해당 OS에 맞는 JDK(Java Development Kit)를 설치합시다. 

- JDK 설치 링크 : <https://www.oracle.com/java/technologies/javase-downloads.html>

In [5]:
from konlpy.tag import Hannanum
hannanum = Hannanum()

kor_text = '밤에 귀가하던 여성에게 범죄를 시도한 대 남성이 구속됐다서울 제주경찰서는 \
            상해 혐의로 씨를 구속해 수사하고 있다고 일 밝혔다씨는 지난달 일 피해 여성을 \
            인근 지하철 역에서부터 따라가 폭행을 시도하려다가 도망간 혐의를 받는다피해 \
            여성이 저항하자 놀란 씨는 도망갔으며 신고를 받고 주변을 수색하던 경찰에 \
            체포됐다피해 여성은 이 과정에서 경미한 부상을 입은 것으로 전해졌다'

print(hannanum.morphs(kor_text))

['밤', '에', '귀가', '하', '던', '여성', '에게', '범죄', '를', '시도', '하', 'ㄴ', '대', '남성', '이', '구속됐다서울', '제주경찰', '서는', '상하', '어', '혐의', '로', '씨', '를', '구속해', '수사', '하고', '있', '다', '고', '일', '밝혔다씨', '는', '지난달', '일', '피하', '어', '여성', '을', '인근', '지하철', '역', '에서부터', '따르', '아', '가', '아', '폭행', '을', '시도', '하', '려', '다가', '도망가', 'ㄴ', '혐의', '를', '받는다피해', '여성', '이', '저항', '하', '자', '놀라', 'ㄴ', '씨', '는', '도망가', '아며', '신고', '를', '받', '고', '주변', '을', '수색', '하', '던', '경찰', '에', '체포됐다피해', '여성', '은', '이', '과정', '에서', '경미한', '부상', '을', '입', '은', '것', '으로', '전하', '어', '지', '었다']


### 1.2 Kkma 사용하기

이번에는 Kkma 클래스를 사용해봅시다. 사용 방법은 똑같습니다.

In [6]:
from konlpy.tag import Kkma
kkma = Kkma()

print(kkma.morphs(kor_text))

['밤', '에', '귀가', '하', '더', 'ㄴ', '여성', '에게', '범죄', '를', '시도', '하', 'ㄴ', '대', '남성', '이', '구속', '되', '었', '다', '서울', '제주', '경찰서', '는', '상해', '혐의', '로', '씨', '를', '구속', '하', '어', '수사', '하', '고', '있', '다고', '일', '밝히', '었', '다', '씨', '는', '지난달', '일', '피해', '여성', '을', '인근', '지하철', '역', '에서', '부터', '따라가', '폭행을', '시도', '하', '려', '다그', '아', '도망가', 'ㄴ', '혐의', '를', '받', '는', '다', '피해', '여성', '이', '저항', '하', '자', '놀라', 'ㄴ', '씨', '는', '도망가', '었', '으며', '신고', '를', '받', '고', '주변', '을', '수색', '하', '더', 'ㄴ', '경찰', '에', '체포', '되', '었', '다', '피해', '여성', '은', '이', '과정', '에서', '경미', '하', 'ㄴ', '부상', '을', '입', '은', '것', '으로', '전하', '어', '지', '었', '다']


위의 Hannanum과 다른것이 보이시죠? 


### 1.3 Komoran 사용하기

이번에는 Komoran 형태소 분석기를 사용해봅시다.

In [6]:
from konlpy.tag import Komoran
komoran = Komoran()

print(komoran.morphs(kor_text))

['밤', '에', '귀가', '하', '던', '여성', '에게', '범죄', '를', '시도', '하', 'ㄴ', '대', '남성', '이', '구속', '되', '었', '다', '서울', '제주', '경찰서', '는', '상해', '혐의', '로', '씨', '를', '구속', '하', '아', '수사', '하', '고', '있', '다고', '일', '밝히', '었', '다', '씨', '는', '지난달', '일', '피해', '여성', '을', '인근', '지하철', '역', '에서부터', '따라가', '아', '폭행', '을', '시도', '하', '려다가', '도망가', 'ㄴ', '혐의', '를', '받', '는다', '피하', '아', '여성', '이', '저항', '하', '자', '놀라', 'ㄴ', '씨', '는', '도망가', '았', '으며', '신고', '를', '받', '고', '주변', '을', '수색', '하', '던', '경찰', '에', '체포', '되', '었', '다', '피하', '아', '여성', '은', '이', '과정', '에서', '경미', '하', 'ㄴ', '부상', '을', '입', '은', '것', '으로', '전하', '아', '지', '었', '다']


### 1.4 Okt 사용해보기

마지막으로 OKT 토크나이저도 사용해보아요

In [7]:
from konlpy.tag import Okt
okt = Okt()

print(okt.morphs(kor_text))

['밤', '에', '귀가', '하던', '여성', '에게', '범죄', '를', '시도', '한', '대', '남성', '이', '구속', '됐다', '서울', '제', '주', '경찰서', '는', '상해', '혐의', '로', '씨', '를', '구속', '해', '수사', '하고', '있다고', '일', '밝혔다', '씨', '는', '지난달', '일', '피해', '여성', '을', '인근', '지하철', '역', '에서부터', '따라가', '폭행', '을', '시도', '하려다가', '도망간', '혐의', '를', '받는다', '피해', '여성', '이', '저항', '하자', '놀란', '씨', '는', '도망갔으며', '신고', '를', '받고', '주변', '을', '수색', '하던', '경찰', '에', '체포', '됐다', '피해', '여성', '은', '이', '과정', '에서', '경미한', '부상', '을', '입은', '것', '으로', '전해졌다']


Okt는 확연하게 다른 것이 보여지네요. 

## Step 2. 불용어 추가해보기

이제 불용어를 제거할 건데요, 불용어를 잘 생각해 `stopwords`라는 변수에 넣어줍시다.

In [9]:
stopwords = ['에','는','은','을','했','에게','있','이',
             '의','하','한','다','과','때문','할','수',
             '무단','따른','및','금지','전재','경향신문',
             '기자','는데','가','등','들','파이낸셜','저작',
             '등','뉴스', '에게', '로', '에', '를','으로',
             '하', '었', '였', '되', 'ㄴ', '다', '어', '지',
             '으며', '아', '일', '자', '려다가', '았', 'ㄹ', 'ㄴ다']

In [10]:
len(stopwords)

52

불용어를 제거하는 함수를 만들어봅시다. 

In [11]:
def preprocessing(tokenizer, data):
    text_data = []
    
    for sentence in data:
        temp_data = []
        # 토큰화
        temp_data = tokenizer.morphs(sentence)
        # 불용어 제거
        temp_data = [word for word in temp_data if not word in stopwords]
        text_data.append(temp_data)
    
    text_data = list(map(' '.join, text_data))
    return text_data

그럼 형태소 분석기들을 하나씩 적용해볼까요?

In [12]:
hannanum_data = preprocessing(hannanum, df['news'])

In [13]:
hannanum_data[0]

'파주시청 사진제공파주시 파주시청 사진제공파주시파주파이낸셜뉴스 강근주 기 파 주 시는 관내 취약계층 만가구 대 정부 긴급재난지원금 입금 완료했다파주시민 받 긴급재난지원금 이상 가구 기준 만원 받 게 며 가구 만원 가구 만원 가구 만원이다정부 발표 긴급재난지원금 파주시민 지급 금액 다르 이유 국비지방비 부담 비율 때문이다파주시 이미 모든 시민 경기도파주시 재난기본소득인당 각 만원 지급 하고 시민 국비 지원금 만 지급 며 가구 기준 총 지원 금액 파주시 재난기본소득 만원 경기 도 재난기본소득 만원 정부 긴급재난지원금 만원 총 만원 받 게 된다취약계층 아니 시민 오 월일 부터 소 하고 신용체크카드사 홈페이지 에서 긴급재난지원금 지원 신청 세대주 가족 지원금 일괄 신청 어야 한다한편 파 주 시는 김정기 부시장 단장 긴급재난지원금 추진 태스크포스 구성해 긴급재난지원금 원활 게 지급 도록 지원 ㄴ다 저작권자 파이낸셜뉴스 전재재배포'

제대로 적용되고 있다는 것을  확인할 수 있습니다 :)

다른 형태소 분석기들도 하나씩 해보아요.

In [14]:
kkma_data = preprocessing(kkma, df['news'])

In [15]:
kkma_data[0]

'파주 시청 사진 제공 파주시 파주 시청 사진 제공 파주시 파주 강 근주 파주시 관내 취약 계층 만 가구 대하 정부 긴급 재난 지원금 입금 완료 파주 시민 받 긴급 재난 지원금 인 이상 가구 기준 만원 받 게 며 가구 만원 인 가구 만원 인 가구 만원 정부 발표 긴급 재난 지원금 파주시 민 지급 금액 다르 이유 국비 지방비 부담 비율 파주시 이미 모든 시민 경기도 파주시 재난 기본 소득 당 각 만원 지급 고 시민 국비 지원금 만 지급 며 가구 기준 총 지원 금액 파주시 재난 기본 소득 만원 경기도 재난 기본 소득 만원 정부 긴급 재난 지원금 만원 총 만원 받 게 ㄴ다 취약 계층 아니 시민 오 월일 부터 소지 고 신용 체크 카드 사 홈페이지 에서 긴급 재난 지원금 지원 신청 세대주 가족 지원금 일괄 신청 어야 ㄴ다 한편 파주시 김 정기 부시장 단장 긴급 재난 지원금 추진 태스크 포스 구성 긴급 재난 지원금 원활 게 지급 도록 지원 ㄴ다 저작권자 재 배포'

In [12]:
komoran_data = preprocessing(komoran, df['news'])

In [13]:
komoran_data[0]

'파주 시청 사진 제공 파주시 파주 시청 사진 제공 파주시 파주 파이낸셜뉴스 강 근 주 파주시 관내 취약 계층 만 가구 대하 정부 긴급 재난 지원금 입금 완료 파주시 민이 받 긴급 재난 지원금 인 이상 가구 기준 만원 받 게 며 인 가구 만원 인 가구 만원 인 가구 만원 이다 정부 발표 긴급 재난 지원금 파주시 민 지급 금액 다른 이유 국비 지방비 부담 비율 이다 파주시 이미 모든 시민 경기도 파주시 재난 기본소득 인당 각 만원 지급 고 시민 국비 지원금 만 지급 며 인 가구 기준 총 지원 금액 파주시 재난 기본소득 만원 경기도 재난 기본소득 만원 정부 긴급 재난 지원금 만원 총 만원 받 게 취약 계층 아니 시민 오 월 부터 소지 고 신용 체크카드 사 홈페이지 에서 긴급 재난 지원금 지원 신청 세대주 가족 지원금 일괄 신청 아야 한편 파주시 김정기 부시장 단장 긴급 재난 지원금 추진 태스크포스를 구성 긴급 재난 지원금 원활 게 지급 도록 지원 저작권 파이낸셜뉴스 재 배포'

In [18]:
okt_data = preprocessing(okt, df['news'])

In [19]:
okt_data[0]

'파주 시청 사진 제공 파주시 파주 시청 사진 제공 파주시 파주 파이낸셜뉴스 강 근 주 파주시 관내 취약 계층 만 가구 대해 정부 긴급 재난 원금 입금 완료 했다 파주시민 받는 긴급 재난 지원 금은 인 이상 가구 기준 만원 받게 되며 인 가구 만원 인 가구 만원 인 가구 만원 이다 정부 발표 긴급 재난 원금 파주시민 지급 금액 다른 이유 국비 방비 부담 비율 이다 파주시 이미 모든 시민 경기도 파주시 재난 기본소득 인 당 각 만원 지급 하고 있어 시민 국비 원금 만 지급 하며 인 가구 기준 총 지원 금액 파주시 재난 기본소득 만원 경기도 재난 기본소득 만원 정부 긴급 재난 원금 만원 총 만원 받게 된다 취약 계층 아닌 시민 오는 월일 부터 소지 하고 있는 신용 체크카드 사 홈페이지 에서 긴급 재난 원금 지원 신청 있다 세대주 가족 원금 일괄 신청 해야 한다 한편 파주시 김정기 부시장 단장 하는 긴급 재난 원금 추진 태스크포스 구성 해 긴급 재난 원금 원활하게 지급 될 있도록 지원 저작권 파이낸셜뉴스 전 재재 배포'

## 머신러닝 사용하기

이제 데이터에 대한 전처리가 끝났습니다. 머신러닝 모델을 적용해보도록 해요.

저번과 마찬가지로 **나이브 베이즈 분류기** 모델을 적용해 볼 예정입니다. 

### 필요한 라이브러리 및 모듈 import

우선 머신러닝 모델 적용을 위해 필요한 도구를 import 합니다.

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

훈련 데이터와 테스트 데이터를 분리해줍시다.

우리는 데이터가 hannanum, kkma, komoran, okt 4가지가 있어요! 각각에 대한 train, test 데이터를 만들어줍시다.

In [21]:
# Hannanum 데이터를 훈련 데이터와 테스트 데이터로 분리
h_x_train, h_x_test, h_y_train, h_y_test = train_test_split(
    hannanum_data,
    df['news'],
    random_state = 0)

print("훈련용 데이터의 갯수 : {}".format(len(h_x_train)))
print("테스트용 데이터의 갯수 : {}".format(len(h_x_test)))

훈련용 데이터의 갯수 : 2995
테스트용 데이터의 갯수 : 999


In [22]:
# kkma 데이터를 훈련, 테스트 데이터로 분리
kk_x_train, kk_x_test, kk_y_train, kk_y_test = train_test_split(
    kkma_data,
    df['news'],
    random_state = 0)

print("훈련용 데이터의 갯수 : {}".format(len(kk_x_train)))
print("테스트용 데이터의 갯수 : {}".format(len(kk_x_test)))

훈련용 데이터의 갯수 : 2995
테스트용 데이터의 갯수 : 999


In [15]:
# komoran 데이터를 훈련, 테스트 데이터로 분리
ko_x_train, ko_x_test, ko_y_train, ko_y_test = train_test_split(
    komoran_data,
    df['news'],
    random_state = 0)

print("훈련용 데이터의 갯수 : {}".format(len(ko_x_train)))
print("테스트용 데이터의 갯수 : {}".format(len(ko_x_test)))

훈련용 데이터의 갯수 : 2995
테스트용 데이터의 갯수 : 999


In [24]:
# okt 데이터를 분리
okt_x_train, okt_x_test, okt_y_train, okt_y_test = train_test_split(
    okt_data,
    df['news'],
    random_state = 0)

print("훈련용 데이터의 갯수 : {}".format(len(okt_x_train)))
print("테스트용 데이터의 갯수 : {}".format(len(okt_x_test)))

훈련용 데이터의 갯수 : 2995
테스트용 데이터의 갯수 : 999


텍스트를 숫자로 바꿔주려고 합니다. 이를 위해 전처리로 **TF-IDF** 방법을 사용해봅시다.

각 뉴스 문서를 TF-IDF 벡터로 바꾸고, 이를 통해 나이브 베이즈 분류기로 학습해봅시다.

In [16]:
def learn_neive_bayes(X_train, y_train):
    #단어의 수를 카운트하는 사이킷런의 카운트벡터라이저
    count_vec = CountVectorizer()
    X_train_counts = count_vec.fit_transform(X_train)
    
    # 카운트 벡터라이저의 결과로부터 TF-IDF 결과를 얻습니다.
    tfidf_transformer = TfidfTransformer()
    X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
    
    #나이브베이즈 분류기를 수행
    # X_train은 TF-IDF 벡터, y_train은 레이블입니다.
    clf = MultinomialNB().fit(X_train_tfidf, y_train)
    
    return clf

나이브 베이즈 분류기가 학습되었습니다. 모델이 학습되었다면 테스트를 해야겠죠?

텍스트를 입력하면 자동으로 TF-IDF 벡터로 바꾸는 전처리 함수를 만들어봅시다. 이 함수를 통해 텍스트를 바로 나이브베이지 분류기의 입력으로 사용함으로써 보다 용이하게 테스트할 수 있습니다.

In [20]:
def tfidf_vectorizer(data):
    count_vec = CountVectorizer()
    data_counts = count_vec.transform(data)
    data_tfidf = tfidf_transformer.transform(data_counts)
    return data_tfidf

이제 각 형태소 분류기마다 나이브 베이즈 분류기를 학습시킵시다

In [27]:
hannanum_clf = learn_neive_bayes(h_x_train, h_y_train)

In [28]:
kkma_clf = learn_neive_bayes(kk_x_train, kk_y_train)

In [21]:
komoran_clf = learn_neive_bayes(ko_x_train, ko_y_train)

In [ ]:
okt_clf = learn_neive_bayes(okt_x_train, okt_y_train)

이제 어떤 형태소 분류기가 정확한 값을 도출해내는지 확인해볼까요?

In [1]:
y_pred = hannanm_clf.predict(tfidf_vectorizer(h_x_test))
print(metrics.classification_report(h_y_test, y_pred))

NameError: name 'hannanm_clf' is not defined

In [ ]:
y_pred = kkma_clf.predict(tfidf_vectorizer(kk_x_test))
print(metrics.classification_report(kk_y_test, y_pred))

In [28]:
count_vect = CountVectorizer()
data_counts = count_vect.fit_transform(ko_x_train)
data_counts = count_vect.transform(ko_x_test)

In [22]:
y_pred = komoran_clf.predict(tfidf_vectorizer(ko_x_test))
print(metrics.classification_report(ko_y_test, y_pred))

NotFittedError: Vocabulary not fitted or provided

In [ ]:
y_pred = okt_clf.predict(tfidf_vectorizer(okt_x_test))
print(metrics.classificationi_report(okt_y_test, y_pred))